In [1]:
%matplotlib widget
from bmcs_cross_section.api import MKappa, TShape, MKappaParamsStudy, EC2, ReinfLayer, IShape, CustomShape, EC2ConcreteMatMod, EC2PlateauConcreteMatMod
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
import numpy as np
import bmcs_utils.api as bu

In [2]:
# Concrete C3-HF2-165-4
E_cm_C3 = 43596
f_cm_C3 = 103
fctm_sp = 5.6
fctm_fl = 12.51

In [3]:
# solidian REBAR Ø6 and Ø8
A_phi_6 = 28.2
A_phi_8 = 50.2

In [4]:
def get_dp(f_cm = f_cm_C3, high_kappa = 0.00018, B=200, H=150, A = None, E_cm = E_cm_C3,
           z=None, concrete_law='EC2', carbon_type='rebars',
          setup='4pb', L=2000, grid_layers_n=1, grid_layers_spacing=10, grid_layers_area=[], reinf = 'carbon'):
    # carbon_type: 'rebars' -> solidian REBAR Ø8-CCE
    # carbon_type: 'grid' -> solidian GRID Q95-CCE-38-E5
    
    mc = MKappa(low_kappa = 0, high_kappa = high_kappa)
    
    mc.cs_design.matrix=concrete_law
    
    f_ck = EC2.get_f_ck_from_f_cm(f_cm)
    fctm_fl = EC2.get_f_ctm_fl(f_ck, H)
    if concrete_law == 'EC2 with plateau' or concrete_law == 'EC2':
        mc.cs_design.matrix_.trait_set(f_cm = f_cm)
    elif concrete_law == 'piecewise linear':
        mc.cs_design.matrix_.trait_set(
            eps_cy=EC2.get_eps_c3(f_ck),
            eps_cu=EC2.get_eps_cu3(f_ck),
        )
    
    mc.cs_design.matrix_.trait_set(E_cc=E_cm_C3, 
                                   E_ct=E_cm_C3, 
                                   eps_cr = fctm_fl/E_cm_C3, 
                                   eps_tu = fctm_fl/E_cm_C3,)
        
    mc.cross_section_shape_.B = B
    mc.cross_section_shape_.H = H

    if carbon_type == 'rebars':
        E_nm=158000
        f_t=2500
        if A is None:
            A = 3 * A_phi_8
        if z is None:
            z = 4 + 10
    elif carbon_type == 'grid':
        E_nm=230000
        f_t=3300
        
        spacing = 38
        rov_num = int(B/spacing) + 1
        rov_cs_area = 3.62

        if A is None:
            A = rov_num * rov_cs_area
        if z is None:
            z = 10
       
    d = H - z
    
    if reinf == 'carbon':
        if grid_layers_n == 1:
            bl1 = ReinfLayer(name='rebars', z=z, A=A, matmod='carbon')
            bl1.matmod_.trait_set(E=E_nm, f_t=f_t, factor = 1.)
            mc.cross_section_layout.add_layer(bl1)
        else:
            for i in range(grid_layers_n):
                if grid_layers_area:
                    bl1 = ReinfLayer(name='grid', z=z + i*grid_layers_spacing, A=grid_layers_area[i], matmod='carbon')
                else:
                    bl1 = ReinfLayer(name='grid', z=z + i*grid_layers_spacing, A=A, matmod='carbon')
                bl1.matmod_.trait_set(E=E_nm, f_t=f_t, factor = 1.)
                mc.cross_section_layout.add_layer(bl1)
            A = grid_layers_n * A
    else:
        bl1 = ReinfLayer(name='rebars', z=z, A=A, matmod='steel')
        mc.cross_section_layout.add_layer(bl1)

    rho = A/(B*d)
    print('A=', A)
    print(r'rho =', round(rho * 100, 2), r'%')
    # mechanical reinforcement ratio
#     w_m = A * f_t / (B * H * f_cm)
#     print('w_m:', w_m)

    dp = DeflectionProfile(mc=mc)

    dp.beam_design.L = L
    
    if setup == '4pb':
        dp.beam_design.system = '4pb'
        dp.beam_design.beam_.L_F = L/3
    elif setup == '3pb':
        dp.beam_design.system = '3pb'
    elif setup == 'dist':
        dp.beam_design.system = 'simple_beam_dist_load'
    
    print('l/d =', dp.beam_design.L/d)
    
    return dp

In [19]:
get_dp().interact()

A= 150.60000000000002
rho = 0.55 %
l/d = 14.705882352941176


In [4]:
# Ductility study
# Beam: solidian REBAR Ø8-CCE
# failure on concrete compression
dp = get_dp(high_kappa = 0.0001, B=200, H=400, A = 2 * A_phi_8, concrete_law = 'EC2', L=1300) # 'EC2 with plateau', 'piecewise linear'

bl1 = ReinfLayer(name='rebars', z=30, A=1 * (3.14*8)**2, matmod='steel')
# bl1.matmod_.trait_set(E=E_nm, f_t=f_t, factor = 1.)
dp.mc.cross_section_layout.add_layer(bl1)

dp.interact()

A= 100.4
rho = 0.13 %
l/d = 3.3678756476683938


In [5]:
# Ductility study 2
# Beam: solidian REBAR Ø8-CCE
# failure on concrete compression
dp = get_dp(high_kappa = 0.0001, B=200, H=400, A = 2 * A_phi_8, concrete_law = 'EC2', L=1300) # 'EC2 with plateau', 'piecewise linear'

bl1 = ReinfLayer(name='rebars', z=5, A=A_phi_8, matmod='carbon')
bl1.matmod_.trait_set(E=158000, f_t=2500/4)
dp.mc.cross_section_layout.add_layer(bl1)

dp.interact()

A= 100.4
rho = 0.13 %
l/d = 3.3678756476683938


In [6]:
# Ductility study 3 on steel (over reinforced steel cross sections also have ductile behaviour if concrete is high)
dp = get_dp(f_ck = 40, high_kappa = 0.00002, B=200, H=400, A = 7*3.14*16**2, concrete_law = 'EC2', L=1300, reinf = 'steel') # 'EC2 with plateau', 'piecewise linear'
dp.interact()

A= 5626.88
rho = 7.29 %
l/d = 3.3678756476683938


In [8]:
# Beam: solidian REBAR Ø8-CCE
# failure on concrete compression
dp = get_dp(high_kappa = 0.00025, B=200, H=100, A = 10 * A_phi_8, concrete_law = 'EC2', L=1300) # 'EC2 with plateau', 'piecewise linear'

dp.interact()

A= 502.0
rho = 2.92 %
l/d = 15.116279069767442


In [8]:
# Slab: solidian GRID Q95-CCE-38-E5
# disb load

dp = get_dp(high_kappa =0.0002, B=200, H=100, carbon_type = 'grid', L=1300, setup='dist')

dp.interact()

# max load 29*1.3 = 37.7 kN

A= 21.72
rho = 0.12 %
l/d = 14.444444444444445


In [71]:
# Slab: solidian GRID Q95-CCE-38-E5
# tensile failure

dp = get_dp(high_kappa =0.0003, B=200, H=100, carbon_type = 'grid', L=1300)

dp.interact()
# max load 14*2 = 28 kN

A= 21.72
rho = 0.12 %
l/d = 14.444444444444445


In [72]:
# Slab: solidian GRID Q95-CCE-38-E5
# concrete failure

dp = get_dp(high_kappa =0.0003, B=200, H=100, carbon_type = 'grid', L=1300, grid_layers_n=2, grid_layers_spacing=10)

dp.interact()
# max load 24*2 = 48 kN

A= 43.44
rho = 0.24 %
l/d = 14.444444444444445


## Ductility study

In [5]:
# steel example
dp = get_dp(high_kappa = 0.00013, B=150, H=200, L=1300, reinf='steel') # 'EC2 with plateau', 'piecewise linear'
dp.interact()

A= 150.60000000000002
rho = 0.54 %
l/d = 6.989247311827957


In [80]:
# I section

dp = get_dp(high_kappa =0.00012, A = 2 * A_phi_6, L=1300)

dp.mc.cross_section_shape = 'I-shape'
dp.mc.cross_section_shape_.H = 200
dp.mc.cross_section_shape_.B_w = 50
dp.mc.cross_section_shape_.B_f_bot = 50
dp.mc.cross_section_shape_.B_f_top = 150
dp.mc.cross_section_shape_.H_f_bot = 50
dp.mc.cross_section_shape_.H_f_top = 50

dp.interact()

A= 56.4
rho = 0.21 %
l/d = 9.558823529411764


In [81]:
# Rect section to compare with I

dp = get_dp(high_kappa = 0.00025, B=150, H=200, A = 2 * A_phi_6, L=1300) # 'EC2 with plateau', 'piecewise linear'

dp.interact()

A= 56.4
rho = 0.2 %
l/d = 6.989247311827957


In [7]:
# Ductile carbon beam
# grid_layers_area=list(1*np.linspace(0.1, 1, n))
# A =21.72
n=6
# dp = get_dp(high_kappa = 0.00013, B=150, H=200, L=1300, grid_layers_n=n, grid_layers_spacing=10, grid_layers_area=list(reversed([1, 3, 5, 8, 15, 30])))
dp = get_dp(high_kappa = 0.00013, B=150, H=200, L=1300, grid_layers_n=n, grid_layers_spacing=10, grid_layers_area=list([1, 3, 5, 8, 15, 30]))

# dp = get_dp(high_kappa = 0.00013, B=150, H=200, L=1300, carbon_type = 'grid', A=20)

# dp = get_dp(high_kappa = 0.00013, B=150, H=200, L=1300)

dp.interact()

A= 903.6000000000001
rho = 3.24 %
l/d = 6.989247311827957
